# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch==1.4.0
import torch
print(torch.__version__)

     |████████████████████████████████| 753.4 MB 6.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
1.4.0


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

In [ ]:
!pip install stanfordnlp
import stanfordnlp
stanfordnlp.download('en')

     |████████████████████████████████| 158 kB 28.3 MB/s 
Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)


In [ ]:
from xml.etree import cElementTree as ET
import pandas as pd
import string
import re

In [ ]:
# root_dir = "/content/drive/MyDrive/Topic 5/"
# root_dir = "/content/drive/MyDrive/Acads/4-1/NLP/NLP Project/Topic 5/"
root_dir = "/content/drive/MyDrive/4-1/NLP/Assignment1/Topic 5/"
# laptop_train_path = "Train/SemEval'14-ABSA-TrainData_v2 & AnnotationGuidelines/Laptop_Train_v2.xml"
laptop_train_path = "Train/SemEval_14-ABSA-TrainData_v2 _ AnnotationGuidelines/Laptop_Train_v2.xml"
# restaurants_train_path = "Train/SemEval'14-ABSA-TrainData_v2 & AnnotationGuidelines/Restaurants_Train_v2.xml"
restaurants_train_path = "Train/SemEval'14-ABSA-TrainData_v2 _ AnnotationGuidelines/Restaurants_Train_v2.xml"
laptop_test_1_path = "Test1/ABSA_TestData_PhaseA/ABSA_TestData_PhaseA/Laptops_Test_Data_PhaseA.xml"
laptop_test_2_path = "Test2/ABSA_TestData_PhaseB/Laptops_Test_Data_phaseB.xml"
restaurants_test_1_path = "Test1/ABSA_TestData_PhaseA/ABSA_TestData_PhaseA/Restaurants_Test_Data_PhaseA.xml"
restaurants_test_2_path = "Test2/ABSA_TestData_PhaseB/Restaurants_Test_Data_phaseB.xml"

# Data Extraction & Cleaning

In [ ]:
def clean_data(data, categorize = True):
    if not categorize:
        data = data.drop(['aspect_categories'], axis=1)
    return data.dropna()

def xml_to_df(path, clean = False, categorize = True, use_sentence_id_as_id = True):
    columns = ['sentence_id', 'text','aspect_terms','aspect_categories']
    if use_sentence_id_as_id:
        df = pd.DataFrame(columns = columns[1:])
    else:
        df = pd.DataFrame(columns = columns)
    
    tree = ET.parse(path)
    root = tree.getroot()

    for page in root.findall('sentence'):
        if use_sentence_id_as_id:
            temp = [page[0].text]
        else:
            temp = [page.attrib["id"], page[0].text]
        for i in range(1,len(page)):
            temp.append([x.attrib for x in page[i]])
        temp+=[None]*(3-len(page))
        if use_sentence_id_as_id:
            df.loc[int(page.attrib["id"])] = temp
        else:
            df.loc[len(df)] = temp
            

    if (clean):
        return clean_data(df, categorize)
    else:
        return df

In [ ]:
laptop_train_data = xml_to_df(root_dir+laptop_train_path, clean = True, categorize=False)
laptop_train_data.head()

In [ ]:
# Split into test and train
laptop_train_data_test = laptop_train_data.sample(frac = 0.1)
laptop_train_data = laptop_train_data.drop(laptop_train_data_test.index)

# Preprocessing

In [ ]:
# Remove Punctuations - {, . "" ' ;}

# Find Aspect Terms

In [ ]:
nlp = stanfordnlp.Pipeline()

In [ ]:
def extract_attributes(text):
    doc = nlp(text)
    parsed_text = {'word':[], 'lemma':[], 'pos':[], 'dep':[], 'gov':[], 'gov_pos':[]}
    for sent in doc.sentences:
        for wrd in sent.words:
            parsed_text['word'].append(wrd.text)
            parsed_text['lemma'].append(wrd.lemma)
            parsed_text['pos'].append(wrd.pos)
            parsed_text['dep'].append(wrd.dependency_relation)
    for sent in doc.sentences:
        for wrd in sent.words:        
            if wrd.governor == 0:
                parsed_text['gov'].append("ROOT")
                parsed_text['gov_pos'].append("-")
            else:   
                parsed_text['gov'].append(parsed_text['word'][wrd.governor-1])
                parsed_text['gov_pos'].append(parsed_text['pos'][wrd.governor-1])
    
    return pd.DataFrame(parsed_text)

In [ ]:
text = "The display on this computer is the best I've seen in a very long time, the battery life is very long and very convenient."
extract_attributes(text)

In [ ]:
def find_aspect_terms(data):
    aspect_terms = []
    only_aspects = []
        
    for i in data.index:
        aspect_terms_df = pd.DataFrame(columns = ["gen_aspect_terms", "sentiment_words"])
        only_aspects_df = pd.DataFrame(columns = ["gen_aspect_terms"])
        line = data['text'][i]

        extracted_attributes_df = extract_attributes(line)
        
        #Rules:
        for j in extracted_attributes_df.index:
            if extracted_attributes_df['dep'][j] == "nsubj":
                if extracted_attributes_df['gov_pos'][j] in ["JJ", "JJR", "JJS"] and extracted_attributes_df['pos'][j] in ["NN", "NNS", "NNP"]:
                    aspect_terms_df.loc[len(aspect_terms_df)] = [extracted_attributes_df['word'][j], [extracted_attributes_df['gov'][j]]] 
                
                elif extracted_attributes_df['gov_pos'][j] in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"] and extracted_attributes_df['pos'][j] in ["NN", "NNS", "NNP"]:
                    aspect_terms_df.loc[len(aspect_terms_df)] = [extracted_attributes_df['word'][j], [extracted_attributes_df['gov'][j]]]
            
            elif extracted_attributes_df['dep'][j] == "amod":
                if extracted_attributes_df['gov_pos'][j] in ["NN", "NNS", "NNP"] and extracted_attributes_df['pos'][j] in ["JJ", "JJR", "JJS"]:
                    aspect_terms_df.loc[len(aspect_terms_df)] = [extracted_attributes_df['gov'][j], [extracted_attributes_df['word'][j]]]
                
                elif extracted_attributes_df['gov_pos'][j] in ["NN", "NNS", "NNP"] and extracted_attributes_df['pos'][j] in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]:
                    aspect_terms_df.loc[len(aspect_terms_df)] = [extracted_attributes_df['gov'][j], [extracted_attributes_df['word'][j]]]
            
            elif extracted_attributes_df['dep'][j] == "dobj":
                if extracted_attributes_df['gov_pos'][j] in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"] and extracted_attributes_df['pos'][j] in ["NN", "NNS", "NNP"]:
                    aspect_terms_df.loc[len(aspect_terms_df)] = [extracted_attributes_df['word'][j], [extracted_attributes_df['gov'][j]]]
            
            elif extracted_attributes_df['dep'][j] == "nmod":
                # Both are aspects
                if extracted_attributes_df['gov_pos'][j] in ["NN"] and extracted_attributes_df['pos'][j] in ["NN", "NS"]:
                    only_aspects_df.loc[len(only_aspects_df)] = [[extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j]]]
                
                elif extracted_attributes_df['gov_pos'][j] in ["JJ"] and extracted_attributes_df['pos'][j] in ["NN"]:
                    aspect_terms_df.loc[len(aspect_terms_df)] = [extracted_attributes_df['word'][j], [extracted_attributes_df['gov'][j]]]
            
            elif extracted_attributes_df['dep'][j] == "acl":
                if extracted_attributes_df['gov_pos'][j] in ["NN"] and extracted_attributes_df['pos'][j] in ["JJ"]:
                    aspect_terms_df.loc[len(aspect_terms_df)] = [extracted_attributes_df['gov'][j], [extracted_attributes_df['word'][j]]]
                
                elif extracted_attributes_df['gov_pos'][j] in ["NNS"] and extracted_attributes_df['pos'][j] in ["VBP"]:
                    aspect_terms_df.loc[len(aspect_terms_df)] = [extracted_attributes_df['gov'][j], [extracted_attributes_df['word'][j]]]
            
            elif extracted_attributes_df['dep'][j] == "conjA":
                # Both are aspects
                if extracted_attributes_df['gov_pos'][j] in ["NN"] and extracted_attributes_df['pos'][j] in ["NN", "NNS", "NNP"]:
                    only_aspects_df.loc[len(only_aspects_df)] = [[extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j]]]

                elif extracted_attributes_df['gov_pos'][j] in ["NN", "NNS", "NNP"] and extracted_attributes_df['pos'][j] in ["JJ"]:
                    aspect_terms_df.loc[len(aspect_terms_df)] = [extracted_attributes_df['gov'][j], [extracted_attributes_df['word'][j]]]
                
                elif extracted_attributes_df['gov_pos'][j] in ["NN", "NNS", "NNP"] and extracted_attributes_df['pos'][j] in ["VBZ"]:
                    aspect_terms_df.loc[len(aspect_terms_df)] = [extracted_attributes_df['gov'][j], [extracted_attributes_df['word'][j]]]

            elif extracted_attributes_df['dep'][j] == "compund":
                # Both are aspects
                if extracted_attributes_df['gov_pos'][j] in ["NN"] and extracted_attributes_df['pos'][j] in ["NN"]:
                    only_aspects_df.loc[len(aspect_terms_df)] = [[extracted_attributes_df['gov'][j], extracted_attributes_df['word'][j]]]

        aspect_terms_row = []
        for ind in aspect_terms_df.index:
            aspect_terms_row.append({aspect_terms_df["gen_aspect_terms"][ind]:aspect_terms_df["sentiment_words"][ind]})           
        aspect_terms.append([i,aspect_terms_row])

        only_aspects_row = []
        for ind in only_aspects_df.index:
            only_aspects_row.append(only_aspects_df["gen_aspect_terms"][ind])           
        only_aspects.append([i,only_aspects_row])

    aspect_terms = pd.DataFrame(aspect_terms, columns = ["sentence_id", "gen_aspect_terms"])
    aspect_terms = aspect_terms.set_index("sentence_id")

    only_aspects = pd.DataFrame(only_aspects, columns = ["sentence_id", "gen_aspect_terms"])
    only_aspects = only_aspects.set_index("sentence_id")
    return aspect_terms, only_aspects

In [ ]:
laptop_train_data_aspect_terms, laptop_train_data_only_aspects = find_aspect_terms(laptop_train_data[:])
print(laptop_train_data_aspect_terms)
print(laptop_train_data_only_aspects)

In [ ]:
def aspect_term_performance(train_data, generated_aspect_terms):
    true_positives = 0
    false_positives = 0
    # false_negatives = 0

    for i in train_data.index:
        # List of actual aspect_terms
        actual_aspect_terms_i = [x['term'] for x in train_data["aspect_terms"][i]]
        generated_aspect_terms_at_i = [list(x.keys())[0] for x in generated_aspect_terms["gen_aspect_terms"][i]]

        for terms in generated_aspect_terms_at_i:
            found = False
            for actual_terms in actual_aspect_terms_i:
                if re.search(terms,actual_terms):
                    true_positives += len(terms.split())
                    found = True
                    break
            if not found:
                false_positives += len(terms.split())

    ratios = {"Precision": true_positives / (true_positives + false_positives)}
    return ratios

In [ ]:
laptop_train_data_ratios = aspect_term_performance(laptop_train_data, laptop_train_data_aspect_terms)
print(laptop_train_data_ratios)

# Train the Model
Make a frequency dictionary for all the Sentiment Words

In [ ]:
def train(train_data, generated_aspect_terms):
    '''
    Both are dataframes
    '''
    # Do preprocessing if needed
    sentiment_words_frequency = {}
    for i in train_data.index:
        
        generated_aspect_terms_at_i = generated_aspect_terms.loc[i]
        
        # x is a dictionary of an actual aspect term
        for x in train_data['aspect_terms'][i]:
            # Actual
            aspect_term = x['term']
            
            if x['polarity'] == 'positive':
                polarity = 1
            
            elif x['polarity'] == 'negative':
                polarity = -1
            
            else:
                polarity = 0
            
            for y in generated_aspect_terms_at_i["gen_aspect_terms"]:
                aspect_term = list(y.keys())[0]
                sentiment_words = y[aspect_term]
                
                for sentiment_word in sentiment_words:

                    if sentiment_word in list(sentiment_words_frequency.keys()): 
                        sentiment_words_frequency[sentiment_word]['frequency'] += 1
                        sentiment_words_frequency[sentiment_word]['polarity'] += (polarity)
                    
                    else:
                        sentiment_words_frequency[sentiment_word] = {'frequency':1, 'polarity':polarity}

    for key in list(sentiment_words_frequency.keys()):
        sentiment_words_frequency[key] = (sentiment_words_frequency[key]['polarity']/sentiment_words_frequency[key]['frequency'])

    return sentiment_words_frequency

In [ ]:
laptop_sentiment_words_frequency_train = train(laptop_train_data, laptop_train_data_aspect_terms)
print(len(laptop_sentiment_words_frequency_train))
print(laptop_sentiment_words_frequency_train)

696
{'good': 0.7857142857142857, 'said': -0.5, 'do': -0.8888888888888888, 'high': 0.47619047619047616, 'other': 0.2328767123287671, 'Great': 0.95, 'many': 0.2978723404255319, 'great': 0.768595041322314, 'next': -0.4666666666666667, 'dark': -0.375, 'steady': -0.75, 'same': -0.22916666666666666, 'blue': -0.14285714285714285, 'soft': 0.0, 'clever': 0.0, 'multi-touch': 0.6666666666666666, 'make': 0.375, 'large': 0.5, 'external': 0.15, 'entire': 0.3333333333333333, 'works': 1.0, 'incredible': 1.0, 'crashed': -0.8571428571428571, 'pure': 1.0, 'painless': 1.0, 'added': 0.5, 'slow': -0.8888888888888888, 'several': 0.0, 'best': 0.6, 'newer': 0.3333333333333333, 'numerous': -1.0, 'AMERICAN': -1.0, 'running': -0.5, 'expired': -0.625, 'cost': -0.6666666666666666, 'hard': -0.10752688172043011, 'certain': -0.1111111111111111, 'Technical': 0.0, 'slick': 0.0, 'defrags': 0.0, 'open': 0.5714285714285714, 'Amazing': 1.0, 'underscores': 0.5, 'died': -0.6, 'work': -0.75, 'stopped': -0.7333333333333333, 'ne

# Test the Model

In [ ]:
def predict(test_data, sentiment_words_frequency):
    generated_aspect_terms,_ = find_aspect_terms(test_data)
    all_unk = 0
    for i in generated_aspect_terms.index:
        for x in generated_aspect_terms["gen_aspect_terms"][i]:
            aspect_term = list(x.keys())[0]
            sentiment_words = x[aspect_term]
            unk = 0
            polarity = 0
            for word in sentiment_words:
                if word not in list(sentiment_words_frequency.keys()):
                    unk += 1
                else:
                    polarity += sentiment_words_frequency[word]
            if (len(sentiment_words) - unk != 0):
                x[aspect_term] = polarity/(len(sentiment_words)-unk)
            else:
                x[aspect_term] = 0
            all_unk += unk
    data = pd.concat([test_data, generated_aspect_terms], axis=1, join="inner")
    return data, all_unk

def classify(predictions):
    classes = []
    for i in range(predictions.shape[0]):
        terms_in_classes = []
        aspect_terms_at_i = predictions['gen_aspect_terms'].iloc[i] 
        for terms in aspect_terms_at_i:
            aspect_term = list(terms.keys())[0]
            polarity = terms[aspect_term]
            
            if (polarity < 0):
                sentiment = "negative"
            elif (polarity > 0):
                sentiment = "positive"
            else:
                sentiment = "neutral"

            terms_in_classes.append({aspect_term:sentiment})
        classes.append(terms_in_classes)
    predictions['classified_aspect_terms'] = classes

In [ ]:
laptop_test_1_data = xml_to_df(root_dir+laptop_test_1_path, clean = False, categorize = False, use_sentence_id_as_id = False)
laptop_test_1_data.head()

,sentence_id,text,aspect_terms,aspect_categories
0,892:1,"Boot time is super fast, around anywhere from ...",None,None
1,1144:1,tech support would not fix the problem unless ...,None,None
2,805:2,but in resume this computer rocks!,None,None
3,359:1,Set up was easy.,None,None
4,562:1,Did not enjoy the new Windows 8 and touchscree...,None,None


In [ ]:
laptop_test_1_predictions, laptop_test_1_predictions_unk_count = predict(laptop_test_1_data, laptop_sentiment_words_frequency_train)
classify(laptop_test_1_predictions)
print("Unknown Sentiment Words:", laptop_test_1_predictions_unk_count)
laptop_test_1_predictions.head()

  sentence_id  ...                      gen_aspect_terms
0       892:1  ...        [{'time': 0.9411764705882353}]
1      1144:1  ...                   [{'support': -1.0}]
2       805:2  ...                     [{'computer': 0}]
3       359:1  ...                                    []
4       562:1  ...  [{'functions': 0.05405405405405406}]

[5 rows x 5 columns]
Unknown Sentiment Words: 170


,sentence_id,text,aspect_terms,aspect_categories,gen_aspect_terms,classified_aspect_terms
0,892:1,"Boot time is super fast, around anywhere from ...",None,None,[{'time': 0.9411764705882353}],[{'time': 'positive'}]
1,1144:1,tech support would not fix the problem unless ...,None,None,[{'support': -1.0}],[{'support': 'negative'}]
2,805:2,but in resume this computer rocks!,None,None,[{'computer': 0}],[{'computer': 'neutral'}]
3,359:1,Set up was easy.,None,None,[],[]
4,562:1,Did not enjoy the new Windows 8 and touchscree...,None,None,[{'functions': 0.05405405405405406}],[{'functions': 'positive'}]


In [ ]:
# Save to a .csv file
laptop_test_1_predictions.to_csv(path_or_buf = root_dir+"Test1/ABSA_TestData_PhaseA/ABSA_TestData_PhaseA/"+"Stanford_Pred.csv")

# Performance of Sentiment Classification

In [ ]:
# Split data and do this
def classification_performance(test_data_predicted, test_data_with_sentiments):
    matches = 0
    non_matches = 0
    for i in test_data_predicted.index:
        aspect_terms_i = test_data_predicted["classified_aspect_terms"][i]
        actual_aspect_terms_i = test_data_with_sentiments["aspect_terms"][i]
        for term in aspect_terms_i:
            aspect_term = list(term.keys())[0]
            polarity = term[aspect_term]
            for actual_term in actual_aspect_terms_i:
                if (aspect_term == actual_term['term']):
                    if (polarity == actual_term['polarity']):
                        matches+=1
                    else:
                        non_matches+=1

    return {"Matches":matches, "Non Matches":non_matches, "Ratio":matches/(matches+non_matches)}

In [ ]:
laptop_train_data_test_predictions, laptop_train_data_test_predictions_unk_count  =  predict(laptop_train_data_test, laptop_sentiment_words_frequency_train)
classify(laptop_train_data_test_predictions)
print("Unknown Sentiment Words:", laptop_train_data_test_predictions_unk_count)
laptop_train_data_test_predictions.head()

Unknown Sentiment Words: 49
                                                   text  ...                            classified_aspect_terms
900   Then, within 5 months, the charger crapped out...  ...                           [{'charger': 'neutral'}]
2941  I wish it had a webcam though, then it would b...  ...                                                 []
2759  This laptop meets every expectation and Window...  ...   [{'laptop': 'neutral'}, {'Windows': 'positive'}]
87    It is easy to operate and I have already order...  ...  [{'software': 'positive'}, {'Rolls': 'positive'}]
2451     Im glad that it has such great features in it.  ...  [{'features': 'neutral'}, {'features': 'positi...

[5 rows x 4 columns]
{'Matches': 23, 'Non Matches': 29, 'Ratio': 0.4423076923076923}


In [ ]:
laptop_train_data_test_ratios = classification_performance(laptop_train_data_test_predictions, laptop_train_data_test)
print(laptop_train_data_test_ratios)

# Demo


In [ ]:
def test(string):
  columns = ['text','aspect_terms','aspect_categories']
  dft = pd.DataFrame(columns=columns)
  temp = [string,None,None]
  dft.loc[len(dft)] = temp
  laptop_test_data = dft
  laptop_test_predictions, laptop_test_predictions_unk_count = predict(laptop_test_data, laptop_sentiment_words_frequency_train)
  print(laptop_test_predictions.head())
  classify(laptop_test_predictions)
  print("Unknown Sentiment Words:", laptop_test_predictions_unk_count)
  print("")
  print("Analysis:")
  print(laptop_test_predictions)

In [ ]:
test("It is a good laptop but bad mouse")


                                text  ...                                   gen_aspect_terms
0  It is a good laptop but bad mouse  ...  [{'laptop': 0.7857142857142857}, {'mouse': -0....

[1 rows x 4 columns]
Unknown Sentiment Words: 0

Analysis:
                                text  ...                          classified_aspect_terms
0  It is a good laptop but bad mouse  ...  [{'laptop': 'positive'}, {'mouse': 'negative'}]

[1 rows x 5 columns]


In [ ]:
test("It is super fast and has outstanding graphics.")

                                             text  ...     gen_aspect_terms
0  It is super fast and has outstanding graphics.  ...  [{'graphics': 0.6}]

[1 rows x 4 columns]
Unknown Sentiment Words: 0

Analysis:
                                             text  ...     classified_aspect_terms
0  It is super fast and has outstanding graphics.  ...  [{'graphics': 'positive'}]

[1 rows x 5 columns]


In [ ]:
test("Laptop good, mouse bad")

                     text  ...                                   gen_aspect_terms
0  Laptop good, mouse bad  ...  [{'Laptop': 0.7857142857142857}, {'mouse': -0....

[1 rows x 4 columns]
Unknown Sentiment Words: 0

Analysis:
                     text  ...                          classified_aspect_terms
0  Laptop good, mouse bad  ...  [{'Laptop': 'positive'}, {'mouse': 'negative'}]

[1 rows x 5 columns]


## Shortcomings and improvements

* Compound aspect terms
* Compount sentiment words
* More rules, curent ones don't handle many cases 
* Improve accuracy
* Precision of genration of aspect terms is an approximation
* Can try Ngram model - for 3 cases with aspect-aspect

